# process_cmip6_files.ipynb
Post-processing of CMIP6 data that have been downloaded from ESGF:
1. Regrid to regular lon/lat grid (using CDO's remapcon / remapcon2 conservative interpolation).

B. S. Grandey, 2022

In [1]:
! date

Wed Jan 26 19:41:54 +08 2022


In [2]:
import pathlib

## Base paths

In [3]:
# CMIP6 input data downloaded from ESGF (organised as <variable>/<source_id>_<member_id>/<instance_id>)
in_base = pathlib.Path('~/Data/p22b/CMIP6/').expanduser()
# Output data
out_base = pathlib.Path('~/Data/p22c/CMIP6/').expanduser()
out_base.mkdir(exist_ok=True)

## Function to find available data and regrid using CDO's remapcon / remapcon2 for a given variable, table_id, source-member pair, and experiment

In [4]:
def apply_cdo_regrid(variable='zos',
                     table_id='Omon',
                     source_member='MRI-ESM2-0_r1i1p1f1',
                     experiment='historical',
                     overwrite=False,  # overwrite previously produced files?
                    ):
    print(f'---- {variable} {table_id} {source_member} {experiment} ({variable}/{source_member}/) ----')
    # Identify most appropriate input data directory
    # Search for available input directories
    in_dirs = list(in_base.glob(f'{variable}/{source_member}/CMIP6.*.{experiment}.*.{table_id}.{variable}.*'))
    in_dirs = sorted(in_dirs)
    # Case: no input directories
    if len(in_dirs) == 0:
        print('No suitable input data found')
        return None
    # Case: one suitable input directory
    elif len(in_dirs) == 1:
        in_dir = in_dirs[0]
        print(f'1 suitable input dir: {in_dir.name}')
    # Case: more than one suitable input directory
    else:
        print(f'{len(in_dirs)} suitable input dirs:')
        for d in in_dirs:
            print(f'{d.name} ({len(list(d.glob("*.nc")))} NetCDF file(s))')
        # Select most recently dated version
        vmax = 0  # initialise "max" version as 0
        for d in in_dirs:
            vmax = max(vmax, int(d.name[-8:]))
        for d in in_dirs:
            if str(vmax) not in d.name:
                in_dirs.remove(d)
        # After version selection, does only one input dir remain?
        if len(in_dirs) == 1:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (most recent)')
        # Prefer data that has already been remapped
        else:
            for d in in_dirs:
                if '.gn.' in d.name:
                    in_dirs.remove(d)
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (non-native grid)')
    # Output data directory
    out_dir = out_base.joinpath(f'regrid/{variable}/{source_member}/{in_dir.name}')
    out_dir.mkdir(parents=True, exist_ok=True)
    # Identify input data files
    in_fns = sorted(list(in_dir.glob('*.nc')))
    # Input grid type (informs whether to use remapcon or remapcon2)
    grid_str = ! cdo sinfo {in_fns[0]} | grep "points"
    in_grid_type = grid_str[0].split(':')[1].strip()
    print(f'Input data grid is {in_grid_type}')
    # Loop over input data files
    for in_fn in in_fns:
        # Output filename depends on in_grid_type
        if in_grid_type == 'unstructured':
            out_fn = out_dir.joinpath(f'{in_fn.stem}.remapcon.nc')
        else:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.remapcon2.nc')
        # Has the output file been created before?  If so, do not overwrite (unless overwrite=True)
        if out_fn.exists() and not overwrite:
            print(f'{out_fn.name} already exists')
            continue
        print(f'Writing {out_fn.name}')
        # If grid is unstructured, use CDO's remapcon and sellonlatbox (to shift longitudes)
        if in_grid_type == 'unstructured':
            ! cdo -O -s sellonlatbox,0,360,-90,90 -remapcon,global_1 {in_fn} {out_fn}
        # Otherwise use CDO's remapcon2
        else:
            ! cdo -O -s sellonlatbox,0,360,-90,90 -remapcon2,global_1 {in_fn} {out_fn}
    return 0

## Apply function(s) to variables and experiments of interest

In [5]:
# For variables and experiments of interest, apply regridding function to all available source-member pairs
for variable, table_id in [('zos', 'Omon'),]:
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        for experiment in ['historical', ]:
            temp = apply_cdo_regrid(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment)

---- zos Omon ACCESS-CM2_r1i1p1f1 historical (zos/ACCESS-CM2_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Omon.zos.gn.v20191108
Input data grid is curvilinear
Writing zos_Omon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.remapcon2.nc
---- zos Omon ACCESS-ESM1-5_r1i1p1f1 historical (zos/ACCESS-ESM1-5_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.CSIRO.ACCESS-ESM1-5.historical.r1i1p1f1.Omon.zos.gn.v20191115
Input data grid is curvilinear
Writing zos_Omon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_185001-201412.remapcon2.nc
---- zos Omon AWI-CM-1-1-MR_r1i1p1f1 historical (zos/AWI-CM-1-1-MR_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.AWI.AWI-CM-1-1-MR.historical.r1i1p1f1.Omon.zos.gn.v20181218
Input data grid is unstructured
Writing zos_Omon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_185001-185012.remapcon.nc
Writing zos_Omon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_185101-186012.remapcon.nc
Writing zos_Omon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_186

Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_194501-194912.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_195001-195412.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_195501-195912.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_196001-196412.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_196501-196912.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_197001-197412.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_197501-197912.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_198001-198412.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_198501-198912.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_199001-199412.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_199501-199912.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_200001-200412.remapcon2.nc
Writing zos_Omon_E3SM-1-1_historical_r1i1p1f1_gr_200501-200912.remapcon2.nc
Writing zos_

Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193201-193212.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193301-193312.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193401-193412.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193501-193512.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193601-193612.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193701-193712.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193801-193812.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_193901-193912.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_194001-194012.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_194101-194112.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_194201-194212.remapcon2.nc
Writing zos_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_194301-

Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186201-186212.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186301-186312.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186401-186412.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186501-186512.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186601-186612.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186701-186712.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186801-186812.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_186901-186912.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_187001-187012.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_187101-187112.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_187201-187212.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_187301-187312.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical

Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196501-196512.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196601-196612.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196701-196712.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196801-196812.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_196901-196912.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197001-197012.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197101-197112.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197201-197212.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197301-197312.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197401-197412.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197501-197512.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_197601-197612.remapcon2.nc
Writing zos_Omon_EC-Earth3-CC_historical

...

Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197101-197112.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197201-197212.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197301-197312.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197401-197412.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197501-197512.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197601-197612.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197701-197712.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197801-197812.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_197901-197912.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_198001-198012.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_198101-198112.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_198201-198212.remapcon2.nc
Writing zos_Omon_EC-Earth3_historical_r1i1p1f1_gn_198301-198312.remapcon2.nc

Input data grid is lonlat
Writing zos_Omon_INM-CM4-8_historical_r1i1p1f1_gr1_185001-189912.remapcon2.nc
Writing zos_Omon_INM-CM4-8_historical_r1i1p1f1_gr1_190001-194912.remapcon2.nc
Writing zos_Omon_INM-CM4-8_historical_r1i1p1f1_gr1_195001-199912.remapcon2.nc
Writing zos_Omon_INM-CM4-8_historical_r1i1p1f1_gr1_200001-201412.remapcon2.nc
---- zos Omon INM-CM5-0_r1i1p1f1 historical (zos/INM-CM5-0_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.INM.INM-CM5-0.historical.r1i1p1f1.Omon.zos.gr1.v20190610
Input data grid is lonlat
Writing zos_Omon_INM-CM5-0_historical_r1i1p1f1_gr1_185001-189912.remapcon2.nc
Writing zos_Omon_INM-CM5-0_historical_r1i1p1f1_gr1_190001-194912.remapcon2.nc
Writing zos_Omon_INM-CM5-0_historical_r1i1p1f1_gr1_195001-199912.remapcon2.nc
Writing zos_Omon_INM-CM5-0_historical_r1i1p1f1_gr1_200001-201412.remapcon2.nc
---- zos Omon IPSL-CM6A-LR_r1i1p1f1 historical (zos/IPSL-CM6A-LR_r1i1p1f1/) ----
1 suitable input dir: CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r1i1p1f1.Omon.zo


cdo(1) remapcon2 (Abort): Integration stalled: num_subseg exceeded limit (grid1[1003]: lon1=6.17847 lon2=3.0208 lat1=1.56427 lat2=1.5708)!
Writing zos_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_195501-195912.remapcon2.nc

cdo(1) remapcon2 (Abort): Integration stalled: num_subseg exceeded limit (grid1[1003]: lon1=6.17847 lon2=3.0208 lat1=1.56427 lat2=1.5708)!
Writing zos_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_196001-196412.remapcon2.nc

cdo(1) remapcon2 (Abort): Integration stalled: num_subseg exceeded limit (grid1[1003]: lon1=6.17847 lon2=3.0208 lat1=1.56427 lat2=1.5708)!
Writing zos_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_196501-196912.remapcon2.nc

cdo(1) remapcon2 (Abort): Integration stalled: num_subseg exceeded limit (grid1[1003]: lon1=6.17847 lon2=3.0208 lat1=1.56427 lat2=1.5708)!
Writing zos_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_197001-197412.remapcon2.nc

cdo(1) remapcon2 (Abort): Integration stalled: num_subseg exceeded limit (grid1[1003]: lon1=6.17847 lon2=3.0208 lat

Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_187401-187412.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_187501-187512.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_187601-187612.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_187701-187712.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_187801-187812.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_187901-187912.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_188001-188012.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_188101-188112.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_188201-188212.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_188301-188312.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_188401-188412.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_188501-188512.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_188601-188612.remapcon2.nc
Writing zos_Omon_TaiESM1_

Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_198401-198412.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_198501-198512.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_198601-198612.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_198701-198712.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_198801-198812.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_198901-198912.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_199001-199012.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_199101-199112.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_199201-199212.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_199301-199312.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_199401-199412.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_199501-199512.remapcon2.nc
Writing zos_Omon_TaiESM1_historical_r1i1p1f1_gn_199601-199612.remapcon2.nc
Writing zos_Omon_TaiESM1_

In [6]:
! date

Wed Jan 26 20:55:05 +08 2022
